In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:23:06.701530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:23:06.812659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:23:06.812681: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:23:06.834317: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:23:07.367434: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 12:23:13.336248: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:23:13.338182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:23:13.338356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:23:13.338518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:23:13.338677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 400ms/step - loss: 2.1076 - auc: 0.5983
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 2.0286 - auc: 0.7926
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9581 - auc: 0.8222
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7737 - auc: 0.8414
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5904 - auc: 0.8555
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4696 - auc: 0.8551
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3969 - auc: 0.8562
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3355 - auc: 0.8695
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2287 - auc: 0.9078
Epoch 10/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1073 - auc: 0.9464
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - loss: 1.0657 

1/1 [==============================] - 0s 30ms/step - loss: 0.2125 - auc: 0.9921
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2142 - auc: 0.9923
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2150 - auc: 0.9919
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2084 - auc: 0.9929
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2114 - auc: 0.9920
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2103 - auc: 0.9918
Epoch 94/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2143 - auc: 0.9913
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2148 - auc: 0.9911
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2225 - auc: 0.9905
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2137 - auc: 0.9914
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 350ms/step - loss: 3.3824 - auc: 0.2243
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9337 - auc: 0.5842
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 1.6313 - auc: 0.8335
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7576 - auc: 0.8619
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9037 - auc: 0.8647
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9483 - auc: 0.8677
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 1.9258 - auc: 0.8724
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8285 - auc: 0.8820
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7485 - auc: 0.8849
Epoch 10/100
1/1 [==============================] - 0s 39ms/step - loss: 1.5744 - auc: 0.8980
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2669 - auc: 0.9887
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2710 - auc: 0.9874
Epoch 90/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2729 - auc: 0.9864
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2768 - auc: 0.9858
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2557 - auc: 0.9900
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2600 - auc: 0.9890
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2649 - auc: 0.9878
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2578 - auc: 0.9891
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2581 - auc: 0.9882
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2590 - auc: 0.9882
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 348ms/step - loss: 2.7343 - auc: 0.2883
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7474 - auc: 0.7253
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6856 - auc: 0.8461
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7821 - auc: 0.8690
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8478 - auc: 0.8722
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8055 - auc: 0.8831
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6989 - auc: 0.8921
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5973 - auc: 0.8966
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3942 - auc: 0.9123
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2665 - auc: 0.9162
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.2370 - auc: 0.9902
Epoch 89/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2300 - auc: 0.9914
Epoch 90/100
1/1 [==============================] - 0s 39ms/step - loss: 0.2311 - auc: 0.9909
Epoch 91/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2214 - auc: 0.9920
Epoch 92/100
1/1 [==============================] - 0s 39ms/step - loss: 0.2299 - auc: 0.9913
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2288 - auc: 0.9909
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2205 - auc: 0.9925
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2267 - auc: 0.9911
Epoch 96/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2286 - auc: 0.9910
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2253 - auc: 0.9913
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 517ms/step - loss: 3.4710 - auc: 0.2185
Epoch 2/100
1/1 [==============================] - 0s 39ms/step - loss: 1.9256 - auc: 0.5962
Epoch 3/100
1/1 [==============================] - 0s 39ms/step - loss: 1.6090 - auc: 0.8394
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7191 - auc: 0.8691
Epoch 5/100
1/1 [==============================] - 0s 38ms/step - loss: 1.8121 - auc: 0.8851
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9071 - auc: 0.8803
Epoch 7/100
1/1 [==============================] - 0s 40ms/step - loss: 1.8886 - auc: 0.8842
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8304 - auc: 0.8913
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7480 - auc: 0.8975
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6022 - auc: 0.9076
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 34ms/step - loss: 0.2419 - auc: 0.9909
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2406 - auc: 0.9911
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2369 - auc: 0.9915
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2293 - auc: 0.9923
Epoch 92/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2318 - auc: 0.9921
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2323 - auc: 0.9919
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2239 - auc: 0.9931
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2398 - auc: 0.9905
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2397 - auc: 0.9902
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2308 - auc: 0.9912
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 464ms/step - loss: 1.8897 - auc: 0.6309
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7781 - auc: 0.8143
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7576 - auc: 0.8307
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5598 - auc: 0.8597
Epoch 5/100
1/1 [==============================] - 0s 47ms/step - loss: 1.4080 - auc: 0.8680
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3009 - auc: 0.8770
Epoch 7/100
1/1 [==============================] - 0s 46ms/step - loss: 1.2392 - auc: 0.8792
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2167 - auc: 0.8754
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0945 - auc: 0.9222
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9971 - auc: 0.9508
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2121 - auc: 0.9924
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2147 - auc: 0.9918
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2133 - auc: 0.9923
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2119 - auc: 0.9918
Epoch 92/100
1/1 [==============================] - 0s 36ms/step - loss: 0.2097 - auc: 0.9930
Epoch 93/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2134 - auc: 0.9920
Epoch 94/100
1/1 [==============================] - 0s 44ms/step - loss: 0.2091 - auc: 0.9927
Epoch 95/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2111 - auc: 0.9923
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2105 - auc: 0.9919
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2118 - auc: 0.9914
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 364ms/step - loss: 2.0493 - auc: 0.5302
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7283 - auc: 0.8102
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 1.7995 - auc: 0.8381
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7503 - auc: 0.8551
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6169 - auc: 0.8621
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4424 - auc: 0.8734
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3196 - auc: 0.8818
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2589 - auc: 0.8763
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2308 - auc: 0.8693
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1855 - auc: 0.8771
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 45ms/step - loss: 0.2537 - auc: 0.9873
Epoch 89/100
1/1 [==============================] - 0s 45ms/step - loss: 0.2511 - auc: 0.9880
Epoch 90/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2462 - auc: 0.9883
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2482 - auc: 0.9869
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2534 - auc: 0.9871
Epoch 93/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2536 - auc: 0.9870
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2519 - auc: 0.9877
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2489 - auc: 0.9878
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2556 - auc: 0.9862
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2484 - auc: 0.9876
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [9]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 2.0002 - auc: 0.5507
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7632 - auc: 0.8083
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8127 - auc: 0.8299
Epoch 4/100
1/1 [==============================] - 0s 39ms/step - loss: 1.6800 - auc: 0.8566
Epoch 5/100
1/1 [==============================] - 0s 55ms/step - loss: 1.5558 - auc: 0.8616
Epoch 6/100
1/1 [==============================] - 0s 52ms/step - loss: 1.4009 - auc: 0.8683
Epoch 7/100
1/1 [==============================] - 0s 46ms/step - loss: 1.2946 - auc: 0.8748
Epoch 8/100
1/1 [==============================] - 0s 40ms/step - loss: 1.2584 - auc: 0.8689
Epoch 9/100
1/1 [==============================] - 0s 41ms/step - loss: 1.1970 - auc: 0.8834
Epoch 10/100
1/1 [==============================] - 0s 37ms/step - loss: 1.1181 - auc: 0.9094
Epoch 11/100
1/1 [==============================] - 0s 45ms/step - l

1/1 [==============================] - 0s 43ms/step - loss: 0.2374 - auc: 0.9907
Epoch 89/100
1/1 [==============================] - 0s 44ms/step - loss: 0.2246 - auc: 0.9921
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2241 - auc: 0.9921
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2317 - auc: 0.9914
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2247 - auc: 0.9925
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2346 - auc: 0.9895
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2302 - auc: 0.9908
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2337 - auc: 0.9892
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2296 - auc: 0.9911
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2254 - auc: 0.9918
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [10]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 486ms/step - loss: 2.0455 - auc: 0.5198
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6818 - auc: 0.8149
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 1.7540 - auc: 0.8476
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7471 - auc: 0.8585
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6186 - auc: 0.8724
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4788 - auc: 0.8812
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3348 - auc: 0.8912
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2530 - auc: 0.8905
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2111 - auc: 0.8843
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2116 - auc: 0.8668
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 37ms/step - loss: 0.2369 - auc: 0.9901
Epoch 89/100
1/1 [==============================] - 0s 48ms/step - loss: 0.2439 - auc: 0.9889
Epoch 90/100
1/1 [==============================] - 0s 39ms/step - loss: 0.2333 - auc: 0.9916
Epoch 91/100
1/1 [==============================] - 0s 47ms/step - loss: 0.2438 - auc: 0.9889
Epoch 92/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2377 - auc: 0.9899
Epoch 93/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2355 - auc: 0.9904
Epoch 94/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2291 - auc: 0.9911
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2350 - auc: 0.9900
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2344 - auc: 0.9903
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2441 - auc: 0.9887
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [11]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 528ms/step - loss: 2.1287 - auc: 0.4670
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7149 - auc: 0.8019
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7827 - auc: 0.8390
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7433 - auc: 0.8591
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6179 - auc: 0.8747
Epoch 6/100
1/1 [==============================] - 0s 38ms/step - loss: 1.4726 - auc: 0.8841
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3117 - auc: 0.8967
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2295 - auc: 0.9001
Epoch 9/100
1/1 [==============================] - 0s 42ms/step - loss: 1.1757 - auc: 0.9010
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1690 - auc: 0.8891
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2213 - auc: 0.9919
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2200 - auc: 0.9927
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2190 - auc: 0.9917
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2232 - auc: 0.9914
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2170 - auc: 0.9918
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2204 - auc: 0.9917
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2236 - auc: 0.9915
Epoch 95/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2163 - auc: 0.9920
Epoch 96/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2273 - auc: 0.9897
Epoch 97/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2100 - auc: 0.9932
Epoch 98/100
1/1 [==============================] - 0s 45ms/step - loss: 

In [12]:
m='remove_TorsionFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 347ms/step - loss: 3.5803 - auc: 0.2300
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.0679 - auc: 0.5274
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5870 - auc: 0.8450
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6792 - auc: 0.8746
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8398 - auc: 0.8773
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.9198 - auc: 0.8759
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.9239 - auc: 0.8776
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8817 - auc: 0.8832
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7889 - auc: 0.8886
Epoch 10/100
1/1 [==============================] - 0s 43ms/step - loss: 1.6473 - auc: 0.8993
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2646 - auc: 0.9900
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2593 - auc: 0.9906
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2576 - auc: 0.9902
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2544 - auc: 0.9905
Epoch 92/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2503 - auc: 0.9903
Epoch 93/100
1/1 [==============================] - 0s 49ms/step - loss: 0.2552 - auc: 0.9897
Epoch 94/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2453 - auc: 0.9921
Epoch 95/100
1/1 [==============================] - 0s 46ms/step - loss: 0.2662 - auc: 0.9871
Epoch 96/100
1/1 [==============================] - 0s 46ms/step - loss: 0.2535 - auc: 0.9897
Epoch 97/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2562 - auc: 0.9890
Epoch 98/100
1/1 [==============================] - 0s 44ms/step - loss: 

In [13]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('9feature_remove_one.csv',index=False)
print(de)

                 model       auc
0            all_model  0.948419
1    remove_AtomPairFP  0.931028
2  remove_Connectivity  0.947040
3        remove_Charge  0.946204
4         remove_Kappa  0.948621
5          remove_MAP4  0.943843
6           remove_MOE  0.947862
7      remove_MorganFP  0.946903
8      remove_Topology  0.948239
9     remove_TorsionFP  0.949793
